#GigaChat с использованием Retrieval Augmented Generation(RAG) и self-asking (через промпты) (вторая версия, вопросы задаются после получения основного ответа)

##Установка необходимых библиотек

In [1]:
!pip install gigachain -q
!pip install -U gigachain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 618.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.1/341.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers -q
!pip install faiss-cpu
!pip install pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.1 MB/s eta 0:00:00


##Инициализируем модель

In [3]:
from getpass import getpass
from langchain.chat_models.gigachat import GigaChat

In [4]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

Введите API ключ от GigaChat··········


In [5]:
# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

##Подготовка документа

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

In [9]:
# Загружаем и разбиваем документ
loader = PyPDFLoader("/content/drive/MyDrive/GigaChat_doc/Приказ ФСТЭК России от 25 декабря 2017 г. N 239.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_doc = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 37


##Создание базы данных эмбеддингов

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [17]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru",
        model_kwargs={"device": "cpu"}
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_doc, hf_embeddings_model)

##Создание цепочки

Создадим цепочку, которая предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель, ретривер (база данных) и доп информация на основе вопроса.

###Пример ответа модели без дополнительных вопросов

In [12]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import re

In [13]:
# Создаём простой шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Задаем ретривер
retriever=db.as_retriever()

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
messege = chain.invoke("В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?")
print(messege)

Если установка наложенных средств защиты информации (СрЗИ) невозможна на объекте критической информационной инфраструктуры (КИИ), то необходимо выполнить следующие действия:

1. Определить базовый набор мер по обеспечению безопасности значимого объекта.
2. Выбрать меры по обеспечению безопасности значимых объектов для их реализации.
3. Разработать рабочую (эксплуатационную) документацию на значимый объект.
4. Описать архитектуру подсистемы безопасности значимого объекта.
5. Определить порядок и параметры настройки программных и программно-аппаратных средств, включая средства защиты информации.

При этом следует учитывать, что выбор мер по обеспечению безопасности значимых объектов должен быть основан на категории значимости объекта КИИ и перечне нормативных правовых актов, методических документов и национальных стандартов, которым должен соответствовать значимый объект.


###Пример ответа модели с уточняющим вопросом после ответа

In [15]:
# Создаём шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Задаем ретривер
retriever = db.as_retriever()

# Функция для генерации дополнительных вопросов
def generate_subquestion(question_and_answer):
    main_answer = chain.invoke(f"Посмотри на вопрос и на ответ на него, дополни ответ и напиши его мне. {question_and_answer}")
    return main_answer

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [16]:
question = "В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?"
answer = chain.invoke(question)
main_answer = generate_subquestion(f"{question} {answer}")

print(main_answer)

Если установка наложенных средств защиты информации (СрЗИ) невозможна на объекте критической информационной инфраструктуры (КИИ), то необходимо выполнить следующие действия:

1. Определить базовый набор мер по обеспечению безопасности значимого объекта.
2. Выбрать меры по обеспечению безопасности значимых объектов для их реализации.
3. Разработать рабочую (эксплуатационную) документацию на значимый объект.
4. Описать архитектуру подсистемы безопасности значимого объекта.
5. Определить порядок и параметры настройки программных и программно-аппаратных средств, включая средства защиты информации.
6. Разработать правила эксплуатации программных и программно-аппаратных средств, включая средства защиты информации (правила безопасной эксплуатации).
7. Внедрить организационные и технические меры по обеспечению безопасности значимого объекта и ввести его в действие.
